# Sobol Indices - Boston Housing dataset

In [ ]:
"""
    CRIM - per capita crime rate by town
    ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
    INDUS - proportion of non-retail business acres per town.
    CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
    NOX - nitric oxides concentration (parts per 10 million)
    RM - average number of rooms per dwelling
    AGE - proportion of owner-occupied units built prior to 1940
    DIS - weighted distances to five Boston employment centres
    RAD - index of accessibility to radial highways
    TAX - full-value property-tax rate per $10,000
    PTRATIO - pupil-teacher ratio by town
    B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    LSTAT - % lower status of the population
    MEDV - Median value of owner-occupied homes in $1000's
"""

## 1) Loading Boston Housing dataset, and fitting a Random Forest model

In [1]:
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


def load_boston_dataset():
    data = ds.load_boston()
    x, y = data.data, data.target
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
    print('Boston Housing dataset loaded, shapes :', x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    return (x_train, y_train), (x_test, y_test)

def fit_random_forest(x_train, y_train, x_test, y_test):
    rf = RandomForestRegressor(50, max_depth=5, min_samples_leaf=5)
    rf.fit(x_train, y_train)
    train_score, test_score = 100.0*rf.score(x_train, y_train), 100.0*rf.score(x_test, y_test)
    print('Random Forest performance (train/test) : {:.02f}% / {:.02f}%'.format(train_score, test_score))
    return rf


# Load the Boston Housing dataset
(x_train, y_train), (x_test, y_test) = load_boston_dataset()

# Create and fit a Random Forest model on the dataset
random_forest = fit_random_forest(x_train, y_train, x_test, y_test)

Boston Housing dataset loaded, shapes : (455, 13) (455,) (51, 13) (51,)
Random Forest performance (train/test) : 90.24% / 87.40%


In [2]:
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sobol_indices.dataset_analyser import analyze
# from CVM_indices.CVM_draft import analyze

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

def pie_plot(indices, path=None):
    # Data to plot
    labels = indices.index.values
    fig = plt.figure()
    # fig.suptitle("with size propotional to the sum")
    axes = [fig.add_subplot(221), fig.add_subplot(222), fig.add_subplot(223), fig.add_subplot(224)]
    titles = ["S", "ST", "S_ind", "ST_ind"]
    sums = indices[titles].sum()
    for i, feature in enumerate(titles):
        sizes = indices[feature] / sums[feature]
        # Plot
        wedges, texts, autotext = axes[i].pie(sizes,autopct='%1.1f%%', shadow=False, radius=sums[feature] / sums.max())
        axes[i].title.set_text(feature)
        axes[i].legend(wedges, labels,
          title="variables",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))
    # plt.axis('equal')
    plt.tight_layout()
    if path is not None:
        plt.savefig(path)
    plt.show()


def bar_plot(indices, path=None):
    def _single_bar(ax, feat, pos, c):
        yerr = np.stack([indices[feat] - indices[feat+"_inf"], indices[feat+"_sup"]-indices[feat]])
        ax.bar(np.arange(len(indices)) + pos, indices[feat], 0.30, yerr=yerr, label=feat, color=c)
    ax1 = plt.subplot2grid((2, 1), (0, 0))
    _single_bar(ax1, "S_ind", -0.15, 'orange')
    _single_bar(ax1, "ST_ind", +0.15, 'blue')
    ax1.set_xticks(np.arange(len(indices)))
    ax1.set_xticklabels(indices.index.values)
    ax1.legend()

    ax2 = plt.subplot2grid((2, 1), (1, 0))
    _single_bar(ax2, "S", +0.15, 'red')
    _single_bar(ax2, "S_ind", -0.15, 'orange')
    ax2.set_xticks(np.arange(len(indices)))
    ax2.set_xticklabels(indices.index.values)
    ax2.legend()
    if path is not None:
        plt.savefig(path)
    plt.show()

In [4]:
data = ds.load_boston()
x = data.data
y = data.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
rf = RandomForestRegressor(50, max_depth=5, min_samples_leaf=5)
rf.fit(x_train, y_train)
print("Training set score: %f" % rf.score(x_train, y_train))
print("Test set score: %f" % rf.score(x_test, y_test))

df = pd.DataFrame(x_test, columns=data.feature_names)
# # df["target"] = y_test
# indices = analyze(df, "target")
indices = analyze(rf.predict, df, n=1000, bs=75)
#indices.to_csv('indices_out.csv')
#indices.to_html('indices_out.html')
#indices.to_latex('indices_out.tex')

  0%|          | 0/75 [00:00<?, ?it/s]

Training set score: 0.899265
Test set score: 0.869122


100%|██████████| 75/75 [03:28<00:00,  2.78s/it]


In [7]:
preds = rf.predict(df)
print(preds)
#print(rf.predict)
#print(df)

[23.0380118  30.27561941 16.49056683 22.75576594 16.5115453  21.23502885
 19.0795922  14.76880713 20.94006194 20.83741756 19.20047614 19.43905522
  8.71615667 21.2162074  20.4679702  24.83174093 18.30009474  9.25530366
 46.28429189 15.43203939 23.57402434 23.47706764 15.7505073  24.68590991
 14.92273439 15.92759765 21.71675954 14.11629593 19.37569518 20.77881123
 20.14567721 23.39397849 27.15505978 20.91046687 15.48049946 15.97509283
 35.68471362 21.13828095 20.52800297 23.35408825 19.3611172  28.64723337
 46.29852831 20.2150252  22.9310527  14.0631705  16.28671122 23.39397849
 17.72148201 28.15973699 20.79937728]


In [11]:
print(indices)

            S    ST  S_ind  ST_ind  S_inf  ST_inf  S_ind_inf  ST_ind_inf  \
CRIM     0.24  0.42   0.00    0.01   0.07    0.37        0.0        0.00   
ZN       0.26  0.46   0.00    0.01   0.09    0.40        0.0        0.00   
INDUS    0.22  0.42   0.00    0.01   0.03    0.37        0.0        0.00   
CHAS     0.01  0.01   0.00    0.01   0.00    0.00        0.0        0.00   
NOX      0.22  0.41   0.00    0.01   0.02    0.37        0.0        0.00   
RM       0.67  0.78   0.04    0.14   0.46    0.67        0.0        0.07   
AGE      0.30  0.49   0.00    0.01   0.12    0.45        0.0        0.00   
DIS      0.18  0.34   0.00    0.01   0.06    0.28        0.0        0.00   
RAD      0.11  0.28   0.00    0.01   0.00    0.22        0.0        0.00   
TAX      0.21  0.36   0.00    0.01   0.05    0.30        0.0        0.00   
PTRATIO  0.20  0.36   0.01    0.02   0.03    0.32        0.0        0.01   
B        0.13  0.26   0.00    0.01   0.00    0.21        0.0        0.00   
LSTAT    0.7

In [10]:
print(indices[['S', 'ST', 'S_ind', 'ST_ind']])

            S    ST  S_ind  ST_ind
CRIM     0.24  0.42   0.00    0.01
ZN       0.26  0.46   0.00    0.01
INDUS    0.22  0.42   0.00    0.01
CHAS     0.01  0.01   0.00    0.01
NOX      0.22  0.41   0.00    0.01
RM       0.67  0.78   0.04    0.14
AGE      0.30  0.49   0.00    0.01
DIS      0.18  0.34   0.00    0.01
RAD      0.11  0.28   0.00    0.01
TAX      0.21  0.36   0.00    0.01
PTRATIO  0.20  0.36   0.01    0.02
B        0.13  0.26   0.00    0.01
LSTAT    0.79  0.93   0.05    0.15


In [9]:
# df["target"] = y_test
# df["target"] = np.abs(df["target"] - rf.predict(x_test))

# # indices_err = analyze(df, "target")
# indices_err = analyze(rf.predict, df, n=1000, bs=75)
# indices_err.to_csv('indices_err.csv')
# indices_err.to_html('indices_err.html')
# indices_err.to_latex('indices_err.tex')

# print(indices)
# print(indices_err)
# pie_plot(indices, 'sobol_pie.png')
# bar_plot(indices, 'sobol_bars.png')
# to_latex(indices, 'indices.tex')

# pie_plot(indices_err, os.path.join('results', 'boston', 'sobol_pie.png'))
# bar_plot(indices_err, os.path.join('results', 'boston', 'sobol_bars.png'))
# to_latex(indices_err, os.path.join('results', 'boston', 'indices.tex'))